# Рекомендация тарифов

## Задание проекта

Имеются данные о поведении клиентов, которые уже перешли на тарифы «Смарт» или «Ультра». Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Необходимо постройть модель с максимально большим значением *accuracy*.

## План проекта

1. [**Анализ данных**](#id1)
2. [**Подготовка данных**](#id2)
3. [**Исследование моделей**](#id3)
4. [**Проверка на тестовой выборке**](#id4)
5. [**Проверка модели на адекватность**](#id5)
6. [**Вывод**](#id6)

## Анализ данных <a id='id1'></a>

In [1]:
import os

import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Откроем файл и рассмотрим его:

In [2]:
if os.path.exists('users_behavior.csv'):
    data = pd.read_csv('users_behavior.csv')
elif os.path.exists('/datasets/users_behavior.csv'):
    data = pd.read_csv('/datasets/users_behavior.csv')
else:
    print("Проверьте правильность пути к датасету")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков нет, изменение типа данных не нужно.

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В данных 5 столбцов:

- **calls** - количество звонков
- **minutes** - количество потраченных минут
- **messages** - количество сообщений 
- **mb_used** - количество использованных мегабайтов интернета
- **is_ultra** - наличие тарифа ultra

## Подготовка данных <a id='id2'></a>

Разобьем данные на выборки. Для начала создадим обучающий набор данных, а именно features (признаки) и target (целевой признак). Целевой признак - тариф (is_ultra):

In [4]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части, три выборки: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1, соответственно. 

Таким образом, обучающая выборка - 60% данных, валидационная выборка - 20% данных, тестовая выборка - 20% данных. Используем функцию разделения данных `train_test_split()`.

In [5]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

In [6]:
print(f'Размеры обучающей выборки: {features_train.shape[0]}')
print(f'Размеры валидационной выборки: {features_valid.shape[0]}')
print(f'Размеры тестовой выборки: {features_test.shape[0]}')

Размеры обучающей выборки: 1928
Размеры валидационной выборки: 643
Размеры тестовой выборки: 643


## Исследование моделей <a id='id3'></a>

**1. Модель решающего дерева**

Будем изменять параметры глубины дерева (от 0 до 20) и минимального количества элементов в узле (от 2 до 10).

In [7]:
best_model_tree_accuracy = 0
for depth in range(1, 21):
    for split in range(2,11):
        model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=split)
        model_tree.fit(features_train, target_train)
        predictions = model_tree.predict(features_valid)
    
        accuracy = accuracy_score(target_valid, predictions)
        if accuracy > best_model_tree_accuracy:
            best_model_tree_accuracy = accuracy
            best_model_tree = model_tree
            best_min_samples_tree = split
            best_depth_model_tree = depth

In [8]:
print('Лучший результат у модели решающего дерева')
print('Глубина: ', best_depth_model_tree)
print('Минимальное кол-во элементов в узле: ', best_min_samples_tree)
print('Точность: {0:.3f}'.format(best_model_tree_accuracy))

Лучший результат у модели решающего дерева
Глубина:  3
Минимальное кол-во элементов в узле:  2
Точность: 0.785


**2. Модель случайного леса**

Будем изменять параметры количества деревьев (от 2 до 20) и глубины количества каждого дерева (от 1 до 20):

In [9]:
best_model_forest_accuracy = 0
for est in range(2, 21):
    for depth in range(1, 21):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model_forest.fit(features_train, target_train)
        predictions = model_forest.predict(features_valid)
    
        accuracy = accuracy_score(target_valid, predictions)
        if accuracy > best_model_forest_accuracy:
            best_model_forest_accuracy = accuracy
            best_model_forest = model_forest
            best_depth_model_forest = depth
            best_est_model_forest = est

In [10]:
print('Лучший результат у модели слуйчайного леса:')
print('Глубина: ', best_depth_model_forest)
print('Количество деревьев: ' , best_est_model_forest)
print('Точность: {0:.3f}'.format(best_model_forest_accuracy))

Лучший результат у модели слуйчайного леса:
Глубина:  12
Количество деревьев:  17
Точность: 0.806


**3. Модель логистической регрессии**

In [11]:
model_regression = LogisticRegression(random_state=12345)
model_regression.fit(features_train, target_train)
predictions = model_regression.predict(features_valid)
    
model_regression_accuracy = accuracy_score(target_valid, predictions)

In [12]:
print('Точность: {0:.3f}'.format(model_regression_accuracy))

Точность: 0.711


В итоге:

In [13]:
accuracy = [best_model_tree_accuracy, best_model_forest_accuracy, model_regression_accuracy]
final_sheet = pd.DataFrame(accuracy, columns=['Лучшая точность'], index=['Решающее дерево','Случайный лес','Логистическая регрессия'])
final_sheet

,Лучшая точность
Решающее дерево,0.785381
Случайный лес,0.805599
Логистическая регрессия,0.710731


Таким образом, точность предсказаний у модели случайного леса наивысшая. Известно, что данная модель наиболее медленная, но для нашей задачи скорость не так принципиальна. 

## Проверка модели на тестовой выборке <a id='id4'></a>

In [14]:
predictions = best_model_forest.predict(features_test)
accuracy_test_forest = accuracy_score(target_test, predictions)

print('Точность модели случайного леса на тестовой выборке: {0:.3f}'.format(accuracy_test_forest))

Точность модели случайного леса на тестовой выборке: 0.799


Проверка на тестовой выборке подтверждает, что точность модели случайного леса выше. Таким образом, наша итоговая модель - **модель случайного леса с 17-ю деревьями и глубиной 12** c точностью **0.799**.

## Проверка модели на адекватность <a id='id5'></a>

Сравним точность итоговой модели с точностью случайной модели. Для этого создадим список случайных чисел и вычислим точность на тестовой выборке.

In [15]:
import random
random_list = []
naive_list = []
for i in range(target_valid.shape[0]):
    random_list.append(random.randint(0,1))
    naive_list.append(0)
random_predictions = pd.Series(random_list)
naive_predictions = pd.Series(naive_list)

accuracy_random = accuracy_score(target_test, random_predictions)
accuracy_naive = accuracy_score(target_test, naive_predictions)
print('Точность случайных предсказаний: {0:.3f}'.format(accuracy_random))
print('Точность "наивных" предсказаний: {0:.3f}'.format(accuracy_naive))
print('Точность выбранной модели на тестовой выборке: {0:.3f}'.format(accuracy_test_forest))

Точность случайных предсказаний: 0.481
Точность "наивных" предсказаний: 0.684
Точность выбранной модели на тестовой выборке: 0.799


В итоге, можно сделать вывод, что модель адекватна.

## Вывод <a id='id6'></a>

Мы исследовали показатели точности трех моделей. По итогам получили, что наилучший показатель точности у модели случайного леса, о чем свидетельствуют теоретические данные. 

Наша модель: **модель случайного леса с 17-ю деревьями и глубиной 12** c точностью **0.799**.